In [46]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression as LR
from tqdm import tqdm
import math


# basic libraries
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker

# sklearn
# from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# others
# import multiprocessing as mp
from tqdm import tqdm

# config
import sys
sys.path.append('/home/yusukemh/github/yusukemh/StatisticalDownscaling/writeup')
from config import C_COMMON, C_SINGLE, C_GRID, FILENAME
from util import load_data

# enable autoreload
%load_ext autoreload
%autoreload 2

class Generator(Sequence):
    # Class is a dataset wrapper for better training performance
    def __init__(self, x_set, y_set, batch_size=256):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.indices = np.arange(self.x.shape[0])

    def __len__(self):
        return math.ceil(self.x.shape[0] / self.batch_size)

    def __getitem__(self, idx):
        # inds = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]  # Line A
        inds = self.indices.take(range(idx * self.batch_size, (idx + 1) * self.batch_size), mode='wrap')
        batch_x = self.x[inds]
        batch_y = self.y[inds]
        return batch_x, batch_y

    def on_epoch_end(self):
        np.random.shuffle(self.indices)

class NeuralNetwork():
    
    def __init__(self, model_func, params, columns):
        self.model_func = model_func
        self.params = params
        self.columns = columns
        pass
    
    def evaluate_by_station(self, df_train, df_test, skn):
        df_train_station = df_train[df_train['skn'] == skn]
        df_test_station = df_test[df_test['skn'] == skn]

        # convert to numpy
        x_train, x_test = np.array(df_train_station[self.columns]), np.array(df_test_station[self.columns])
        y_train, y_test = np.array(df_train_station['data_in']), np.array(df_test_station['data_in'])

        # scale the input and output
        x_train, x_test = self.transform_x(x_train, x_test)
        y_train, y_test = self.transform_y(y_train, y_test)
        
        # train the model with retrain_full = True
        history = self.train(x_train, y_train, verbose=0, retrain_full=True)

        # make prediction and scale
        y_pred = self.model.predict(x_test)
        y_pred = self.inverse_transform_y(y_pred)

        # scale y_test
        y_test = self.inverse_transform_y(y_test)
        
        return {
            "skn": skn,
            "rmse_nn": mean_squared_error(y_test, y_pred, squared=False),
            "mae_nn": mean_absolute_error(y_test, y_pred)
        }
        
    
    def evaluate(self, df_train, df_test):
        ret_vals = []
        for skn in tqdm(df_train['skn'].unique()):
            r = self.evaluate_by_station(df_train, df_test, skn)
            ret_vals.append(r)

        return pd.DataFrame(ret_vals)
            
            
    
    def cross_val_predict(self, df, skn, verbose=0, n_folds=5):
        assert 'inner_fold' in df.columns, 'define fold with column name "inner_fold"'
        df_station = df[df['skn'] == skn]
        
        list_ytrue = []
        list_ypred = []
        for k in range(n_folds):
            # split the dataset
            df_train = df_station[df_station['inner_fold'] != k]
            df_test = df_station[df_station['inner_fold'] == k]
            
            # convert to numpy
            x_train, x_test = np.array(df_train[self.columns]), np.array(df_test[self.columns])
            y_train, y_test = np.array(df_train['data_in']), np.array(df_test['data_in'])
            
            # scale the input and output
            x_train, x_test = self.transform_x(x_train, x_test)
            y_train, y_test = self.transform_y(y_train, y_test)
            
            # train the model
            history = self.train(x_train, y_train, verbose=0, retrain_full=False) # to speed up computation for hyperparaemter tuning
            
            # make prediction and scale
            y_pred = self.model.predict(x_test)
            y_pred = self.inverse_transform_y(y_pred)
            # scale y_test
            y_test = self.inverse_transform_y(y_test)
            
            # keep the record
            list_ytrue.extend(y_test)
            list_ypred.extend(y_pred)
        
        # calculate the loss and return
        return {
            "rmse": mean_squared_error(list_ytrue, list_ypred, squared=False),
            "mae": mean_absolute_error(list_ytrue, list_ypred),
            'epochs': len(history.history['loss'])
        }

    def transform_x(self, x_train, x_test):
        scaler = MinMaxScaler()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)
        return x_train, x_test
    
    def transform_y(self, y_train, y_test):
        y_train = np.log(y_train + 1.)
        y_test = np.log(y_test + 1.)

        return y_train, y_test# , scaler
    
    def inverse_transform_y(self, y):
        y = np.power(np.e, y) - 1
        return y
    
    def train(self, x, y, verbose=0, retrain_full=False):
        # split into train and validation
        # strictly speaking, this is not appropriate because scaler fit to the union of train/valid
        x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, shuffle=False)
        # build the model
        self.model, batch_size = self.model_func(**self.params)
        # set up callbacks
        callbacks = [
            EarlyStopping(
                monitor='val_loss',
                min_delta=0,
                patience=20,
                restore_best_weights=True,
            ),
            ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.95,
                patience=10
            )
        ]
        
        # set up the generators
        train_datagen = Generator(x_train, y_train, batch_size)
        valid_datagen = Generator(x_valid, y_valid, batch_size)
        
        history = self.model.fit_generator(
            train_datagen,
            steps_per_epoch=np.ceil(len(x_train)/batch_size),
            validation_data=valid_datagen,
            validation_steps=np.ceil(len(x_valid)/batch_size),
            epochs=int(1e3),
            callbacks=callbacks,
            verbose=0
            
            # epochs=int(1e3),
            # batch_size=batch_size,
            # validation_split=0.2,
            # callbacks=callbacks,
            # verbose=0
        )
        
        if retrain_full:
            epochs = len(history.history['loss'])
            train_datagen = Generator(x, y, batch_size)
            # rebuild the model
            self.model, batch_size = self.model_func(**self.params)
            callbacks = [EarlyStopping(monitor='loss', min_delta=0, patience=1e3, restore_best_weights=True)]
            history = self.model.fit(
                train_datagen,
                steps_per_epoch=np.ceil(len(x) / batch_size),
                epochs=epochs,
                callbacks=callbacks,
                verbose=0,
                #epochs=epochs,
                #validation_split=0,
                #callbacks=callbacks,
                #batch_size=batch_size,
                #verbose=0
            )
        return history        


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
test = np.array([1,2,3,4,5,6,7,8,9,10])
test.take(range(5, 11), mode='wrap')

array([ 6,  7,  8,  9, 10,  1])

In [31]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import L2

def define_model(
    input_dim=20,
    n_units=512,
    activation='selu',#selu
    learning_rate=0.00001,
    loss='mse',
    batch_size=64
):
    inputs = Input(shape=(input_dim))
    x = Dense(units=n_units, activation=activation, kernel_regularizer=L2(l2=0.01))(inputs)
    x = Dropout(rate=0.5)(x)
    x = Dense(units=n_units, activation=activation, kernel_regularizer=L2(l2=0.01))(x)
    x = Dropout(rate=0.5)(x)
    x = Dense(units=n_units, activation=activation, kernel_regularizer=L2(l2=0.01))(x)
    x = Dropout(rate=0.5)(x)# serves as regularization
    outputs = Dense(units=1, activation='softplus')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
        loss=loss,
        metrics=[RootMeanSquaredError()]
    )
    return model, batch_size

In [5]:
from tensorflow.keras.utils import Sequence

In [32]:
columns = C_SINGLE
df_train, df_test = load_data(columns + C_COMMON, FILENAME)

In [49]:
n_model = NeuralNetwork(
    columns = columns,
    params = {'n_units': 274, 'learning_rate': 0.001701746715659, 'input_dim': 16, 'batch_size': 192, 'loss': 'mse'},
    model_func=define_model
)
n_model.cross_val_predict(df_train, skn=54)

/home/yusukemh/.conda/envs/climate/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/home/yusukemh/.conda/envs/climate/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/home/yusukemh/.conda/envs/climate/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/home/yusukemh/.conda/envs/climate/lib/python3.9/site-packages/t

{'rmse': 5.789018970581302, 'mae': 3.572849455263069, 'epochs': 330}

In [ ]:
# make sure cross_val_predict works, and add n_iter for evaluate and evaluate_by_station to take the mean, std of perfoemance

In [47]:
from hyperparameters import NN_PARAMS
ret_vals = []
for item in tqdm(NN_PARAMS):
    station_model = NeuralNetwork(
        columns=columns,
        params=item['params'],
        model_func=define_model
    )
    r = station_model.evaluate_by_station(df_train, df_test, skn=item['skn'])
    ret_vals.append(r)

  0%|          | 0/24 [00:00<?, ?it/s]/home/yusukemh/.conda/envs/climate/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
  4%|▍         | 1/24 [00:35<13:25, 35.03s/it]/home/yusukemh/.conda/envs/climate/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
  8%|▊         | 2/24 [00:54<09:24, 25.65s/it]/home/yusukemh/.conda/envs/climate/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports gener

In [45]:
# n_model.evaluate(df_train, df_test)

  0%|          | 0/24 [00:00<?, ?it/s]/home/yusukemh/.conda/envs/climate/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/1000
1/1 [==============================] - 1s 556ms/step - loss: 8.5489 - root_mean_squared_error: 1.3962 - val_loss: 34.9929 - val_root_mean_squared_error: 5.3319
Epoch 2/1000
1/1 [==============================] - 0s 72ms/step - loss: 28.6793 - root_mean_squared_error: 4.7027 - val_loss: 7.2776 - val_root_mean_squared_error: 0.8584
Epoch 3/1000
1/1 [==============================] - 0s 69ms/step - loss: 7.9652 - root_mean_squared_error: 1.1935 - val_loss: 9.5655 - val_root_mean_squared_error: 1.7451
Epoch 4/1000
1/1 [==============================] - 0s 69ms/step - loss: 9.2526 - root_mean_squared_error: 1.6530 - val_loss: 10.6801 - val_root_mean_squared_error: 2.0449
Epoch 5/1000
1/1 [==============================] - 0s 69ms/step - loss: 10.1796 - root_mean_squared_error: 1.9186 - val_loss: 10.7791 - val_root_mean_squared_error: 2.0742
Epoch 6/1000
1/1 [==============================] - 0s 68ms/step - loss: 10.4265 - root_mean_squared_error: 1.9874 - val_loss: 10.5698 - va

  0%|          | 0/24 [00:06<?, ?it/s]

Epoch 70/1000


KeyboardInterrupt: 

In [48]:
ret_vals

[{'skn': 54.0, 'rmse_nn': 4.8431024466497705, 'mae_nn': 3.0914463170980784},
 {'skn': 79.0, 'rmse_nn': 6.368086866764725, 'mae_nn': 4.105100464089943},
 {'skn': 87.0, 'rmse_nn': 5.208669796473459, 'mae_nn': 3.37957601492572},
 {'skn': 250.0, 'rmse_nn': 2.2157803460396273, 'mae_nn': 1.3270023535467255},
 {'skn': 267.0, 'rmse_nn': 2.2336697521762474, 'mae_nn': 1.4160992786791415},
 {'skn': 296.1, 'rmse_nn': 1.1442526100796995, 'mae_nn': 0.6380257667766676},
 {'skn': 311.0, 'rmse_nn': 1.2396139381603444, 'mae_nn': 0.5953852845453154},
 {'skn': 338.0, 'rmse_nn': 2.672605371750324, 'mae_nn': 1.8492123743180304},
 {'skn': 396.0, 'rmse_nn': 1.2921348638193793, 'mae_nn': 0.743996364183915},
 {'skn': 400.0, 'rmse_nn': 1.3514261195929214, 'mae_nn': 0.8701712177934825},
 {'skn': 406.0, 'rmse_nn': 1.6197638161201364, 'mae_nn': 1.1496237303210841},
 {'skn': 410.0, 'rmse_nn': 1.4479689882305888, 'mae_nn': 0.9798297832562373},
 {'skn': 485.0, 'rmse_nn': 1.85977139749722, 'mae_nn': 1.403804300748385},